In [1]:
import numpy as np
import torch
import pickle
import pandas as pd
from scipy.stats import mannwhitneyu
from lpne.models import DcsfaNmf
from sklearn.metrics import roc_auc_score
#Load the model
model_name = "Final_mt_Model_500_epochs.pt"
model = torch.load(model_name,map_location="cpu")
model.device="cpu"

#Define Load Parameters
FEATURE_LIST = ['X_psd','X_coh','X_gc']
FEATURE_WEIGHT = [10,1,1] #Power features are scaled by 10

#Define and load the data file
c19_epm_file = "C19_EPM_Data.pkl"
dataDict = pickle.load(open(c19_epm_file,"rb"))

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/hpc/home/mk423/.local/lib/python3.7/site-packages/lpne/pipelines/__init__.py:14: UserWarning: Could not load lpne/pipelines/default_params.yaml!
  warnings.warn("Could not load lpne/pipelines/default_params.yaml!")


In [2]:
#Parse data file into relevant variables
X = np.hstack([weight*dataDict[feature] for weight,feature in zip(FEATURE_WEIGHT,FEATURE_LIST)])
y_time = dataDict['y_time']
y_mouse = np.array(dataDict['y_mouse'])
y_condition = np.array(dataDict['y_condition']) #genotype
y_task = dataDict['y_task']
y_roi = dataDict["y_roi"]
y_expDate = dataDict['y_expDate']
y_sex = np.array(dataDict['y_sex'])

In [5]:
np.unique(y_mouse).shape

(17,)

In [4]:
#Project the data
s = model.project(X)

In [5]:
#Get by second projection

#composite network AUC
all_hc_v_task_aucs = model.score(X,y_task,y_mouse,return_dict=True)

coeffs = np.abs(model.classifier[0].weight[0].detach().cpu().numpy())
agg_score = s[:,:3] @ coeffs

results_dict = {
    "agg score":agg_score,
    "net 1 scores":s[:,0],
    "net 2 scores":s[:,1],
    "net 3 scores":s[:,2],
    "mouse":y_mouse,
    "sex":y_sex,
    "roi":y_roi,
    "condition":y_condition,
    "in-task":y_task,
    "time":y_time,
    "expDate":y_expDate,
    "hc_v_task auc":[all_hc_v_task_aucs[mouse][0] for mouse in y_mouse],
}

df = pd.DataFrame.from_dict(results_dict)
#df.to_csv(proj_file)
df.head()

,agg score,net 1 scores,net 2 scores,net 3 scores,mouse,sex,roi,condition,in-task,time,expDate,hc_v_task auc
0,5.505884,0.133097,0.248273,0.086571,Mouse0911,F,NaN,WT,False,1,022021,0.702284
1,3.841483,0.113681,0.160648,0.068322,Mouse0911,F,NaN,WT,False,3,022021,0.702284
2,5.850778,0.132260,0.269141,0.093692,Mouse0911,F,NaN,WT,False,4,022021,0.702284
3,2.884168,0.052870,0.139675,0.051756,Mouse0911,F,NaN,WT,False,5,022021,0.702284
4,2.567462,0.071152,0.111259,0.021368,Mouse0911,F,NaN,WT,False,6,022021,0.702284


In [6]:
#check that the net 1, 2, and 3 scores match
df_check = pd.read_csv("c19_epm_projection.csv")
df_check.head()

,Unnamed: 0,net 1 scores,net 2 scores,net 3 scores,mouse,sex,condition,in-task,time,expDate,hc_v_task auc,net 1 hc_v_task auc,net 2 hc_v_task auc,net 3 hc_v_task auc
0,0,0.133097,0.248273,0.086571,Mouse0911,F,WT,False,1,22021,0.702284,0.619013,0.716532,0.619504
1,1,0.113681,0.160648,0.068322,Mouse0911,F,WT,False,3,22021,0.702284,0.619013,0.716532,0.619504
2,2,0.132260,0.269141,0.093692,Mouse0911,F,WT,False,4,22021,0.702284,0.619013,0.716532,0.619504
3,3,0.052870,0.139675,0.051756,Mouse0911,F,WT,False,5,22021,0.702284,0.619013,0.716532,0.619504
4,4,0.071152,0.111259,0.021368,Mouse0911,F,WT,False,6,22021,0.702284,0.619013,0.716532,0.619504
